In [1]:
import numpy as np 
import pandas as pd 

In [2]:
data_path = '../data_sources/echp/'

In [3]:
df_p = pd.read_stata(data_path+'a_w'+str(1)+'p.dta',convert_categoricals=False)
df_p['pid'] = df_p['pid'].astype('int64')
df_p.set_index(['country','hid','pid'],inplace=True)
df_p = df_p.sort_index()


In [4]:
df_r = pd.read_stata(data_path+'a_w'+str(1)+'rel.dta')


In [5]:
df_h = pd.read_stata(data_path+'a_w'+str(1)+'h.dta',convert_categoricals=False)

In [6]:
df_p.head()

wave  pg001    pg002    pg003  pg004  pg005  pg006  pg007  \
country hid pid                                                               
1       101 1101     1      1  1.08041  1.08041     25     -8     -8     -8   
            1102     1      1  1.08041  1.08041     25     -8     -8     -8   
        201 2102     1      2  1.35196  1.35196     15     -8     -8     -8   
            2103     1      2  1.35196  1.35196     -9     -8     -8     -8   
            2104     1      2  1.35196  1.35196     10     -8     -8     -8   

                  pg008  pd001  ...  pm007a  pm007b  pm007c  pm008  pm010  \
country hid pid                 ...                                         
1       101 1101      1   1962  ...      -8      -8      -8      1     -8   
            1102      1   1964  ...      -8      -8      -8      1     -8   
        201 2102      1   1936  ...      -8      -8      -8      1     -8   
            2103      3   1963  ...      -8      -8      -8      1     -8   
            2104      3   1967  ...      -8      -8      -8      1     -8   

                  pm011  pk001  pk002  pk003  pk004  
country hid pid                                      
1       101 1101     -8      4      4      5      5  
            1102     -8      5      1      6      4  
        201 2102     -8      3      4      5      5  
            2103     -8      3      2      3      2  
            2104     -8      5      3      4      2  

[5 rows x 224 columns]

In [7]:
waves = [x for x in range(1,9)]
df_s = []
pvars = ['wave','hid','pd001','pd003','pd004','pd005','pi100','pi110','pi211m','pi211mg', 'pt022','country','pe001','pe032','pg002','pid']
for w in waves:
	# person file
	df_p = pd.read_stata(data_path+'a_w'+str(w)+'p.dta',convert_categoricals=False)
	df_p = df_p[pvars]
	df_p['pid'] = df_p['pid'].astype('int64')
	df_p = df_p.drop_duplicates(subset=['country','hid','pid'],keep='last')
	df_p.set_index(['country','hid','pid'],inplace=True)
	df_p = df_p.sort_index()	
	# relations
	df_r = pd.read_stata(data_path+'a_w'+str(w)+'rel.dta',convert_categoricals=False)
	df_r = df_r.rename({'pid1':'pid'},axis=1)
	df_r['pid'] = df_r['pid'].astype('int64')
	df_r = df_r[['country','hid','pid','relation']]
	df_r = df_r[df_r['relation'].isin([0,1])]
	df_r = df_r.drop_duplicates(subset=['country','hid','pid'],keep='last')
	df_r.set_index(['country','hid','pid'],inplace=True)
	df = df_r.merge(df_p,left_index=True,right_index=True,how='left')
	# household file
	df_h = pd.read_stata(data_path+'a_w'+str(w)+'h.dta',convert_categoricals=False)
	df_h = df_h[['hid','country','hi100']]
	df_h.set_index(['hid','country'],inplace=True)
	df_h = df_h.sort_index()
	# merging 
	df_m = df.reset_index().merge(df_h,left_on=['hid','country'],right_on=['hid','country'],how='left')
	df_m = df_m.reset_index()
	df_m.set_index(['country','hid','pid'],inplace=True)
	df_s.append(df_m)




In [8]:
df = df_s[0]
for d in df_s[1:]:
	df = df.append(d)

In [9]:
len(df)

497001

Age

In [10]:
df = df.rename({'pd003':'age'},axis=1)
df = df[df['age']>=0]

Income $Y$. We use total net (after tax) household income. 

HI100 TOTAL NET HOUSEHOLD INCOME (DETAILED, NC, TOTAL YEAR PRIOR TO THE SURVEY)

In Italy, currency in liras, but in thousands

In [11]:
df = df.rename({'hi100':'Y'},axis=1)

Drop countries we do not need. Sweden missing because NA. 

In [12]:
df = df[df.index.get_level_values(0).isin([51,2,3,6,9,11])]

Map names of countries

In [13]:
co_map = {51:'DE',2:'DK',3:'NL',6:'FR',9:'IT',11:'SP'}
df['co'] = df.index.get_level_values(0).to_list()
df['co'] = df['co'].replace(co_map)

Only keep dominants in households

In [14]:
df = df[df['relation']==0]

Create a year variables

In [15]:
df['year'] = df['wave'].replace({1:1994,2:1995,3:1996,4:1997,5:1998,6:1999,7:2000,8:2021})

Keep variables of interest

In [16]:
df = df[['wave','year','age','co','Y']]

We will use mostly logs to estimate covariance structures. Therefore, we will normalize incomes using means. 


In [17]:
for c in ['DE','DK','NL','FR','IT','SP']:
	df.loc[df.co==c,'Y'] = df.loc[df.co==c,'Y']/df.loc[df.co==c,'Y'].mean()

In [18]:
df = df[df['Y']>0]

In [19]:

p99 = df['Y'].quantile(0.99)
df = df[df['Y']<=p99]

In [20]:
df['logY'] = np.where((df['Y']>0.0) & (df['Y'].isna()==False),np.log(df['Y']),np.nan)

In [21]:
df['Y'].describe()

count    55082.000000
mean         0.966042
std          0.533180
min          0.000315
25%          0.605625
50%          0.857492
75%          1.213956
max          3.519351
Name: Y, dtype: float64

Simple means by country and year. Still in national currencies. 

In [22]:
df.groupby(['year','co']).mean()['Y'].unstack()

co,DE,DK,FR,IT,NL,SP
year,,,,,,
1994.0,0.902893,0.893645,0.874513,0.831197,0.879031,0.810701
1995.0,0.891001,0.914816,0.918591,0.886458,0.856276,0.868409
1996.0,0.923386,0.931493,0.930152,0.930722,0.878858,0.931103
1997.0,0.964111,0.977993,0.965487,0.947819,0.953462,0.926907
1998.0,0.971816,0.989963,0.998748,1.006032,0.978481,0.960584
1999.0,0.988966,1.018295,1.006490,1.049448,1.034138,1.015213
2000.0,1.026001,1.056225,1.043024,1.068324,1.084338,1.035354
2021.0,1.045274,1.081585,1.064239,1.073784,1.093130,1.073035


A look at what the dataframe looks like

In [23]:
df.head()

wave    year   age  co         Y      logY
country hid   pid                                               
2       7601  76101    1.0  1994.0  42.0  DK  0.829935 -0.186408
        8401  84101    1.0  1994.0  70.0  DK  1.082765  0.079518
        10701 107101   1.0  1994.0  22.0  DK  0.770043 -0.261310
        16501 165101   1.0  1994.0  46.0  DK  1.336161  0.289800
        19901 199101   1.0  1994.0  23.0  DK  0.821379 -0.196771

Saving the output file

In [24]:
df.to_pickle(data_path+'echp_incomes.pkl')
